# 07/19/20 - In order to obtain an idea of how the Einstein radius estimates compare to the true Einstein radius, we compare the overlaps between GAMA and SLACS, which modeled their lenses in a more detailed manner. 
### G136604 was easy because it was one of the overlaps with the candidate samples (LinKS, Zoo).
GAMA ID: 136604  

SDSS ID: J1143−0144  

RA: 175.873 

DEC: -1.74167

Einstein radius :

b_SIE = 1.68 arcsec (Bolton 2008)

theta_e_SIS (Zahid) = 1.345

Theta_e_pm (Auger) = 2.112

theta_e_SIS (Auger) = 1.197

### Now we look at the other (G216398)
# Edit 08/10/20 - In order to compare with the GAMA Spectroscopy candidates, these two should be estimated also with their true source positions.
Beginning of this edit is marked below.
# Edit 08/13/20 - Checking HLA to see if I can find a source redshift from the SLACS lens sample for G3882191.

In [1]:
# import libraries
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from astropy.io import fits
from astropy import constants as const
import astropy.coordinates as coord
import astropy.units as u
from astropy.table import Table
c = const.c.to('km/s').value #c in km/s
h0 = 73.8 #h0 in km/(Mpc*s) Riess2011
from astropy.cosmology import FlatLambdaCDM, z_at_value
cosmo = FlatLambdaCDM(h0,Om0=0.262)

In [2]:
# set datapaths
csv_datapath = '/home/shawn/Desktop/gravitational_lensing_research/Lens_Project_Files/'
png_datapath = '/home/shawn/Desktop/gravitational_lensing_research/Lens_Project_Visuals/'
pdf_datapath = '/home/shawn/Desktop/gravitational_lensing_research/Lens_Project_Visuals/'

In [3]:
# import slacs and lambdar masses
slacs = pd.read_csv(f'{csv_datapath}slacs_gama.csv')
print(slacs.columns)

# import lambdarstellarmasses
# Opening GAMA MagPhys data from G09, G12, and G15 (DR3)
hdul = fits.open('/home/shawn/Desktop/gravitational_lensing_research/FITS_Files/StellarMassesLambdar.fits')  # open a FITS file
hdul.verify('fix')
masses = hdul[1].data  # assume the first extension is a table
#print(masses.columns)
# create dataframe of masses objects to compare to candidates
GAMA_ID = masses.CATAID.byteswap().newbyteorder()
logmstar = masses.logmstar.byteswap().newbyteorder()
logmintsfh = masses.logmintsfh.byteswap().newbyteorder()
logmremnants = masses.logmremnants.byteswap().newbyteorder()
fluxscale = masses.fluxscale.byteswap().newbyteorder()
dellogmintsfh = masses.dellogmintsfh.byteswap().newbyteorder()
z = masses.Z.byteswap().newbyteorder()

#masses_list = list(zip(GAMA_ID, logmstar, logmintsfh, logmremnants, fluxscale))
#masses_list
lambdar_masses = pd.DataFrame(
    {'GAMA_ID' : GAMA_ID,
#     'logmstar' : logmstar, 
     'lambdar_log_mstar' : logmintsfh,
    'lambdar_mstar' : 10**(logmintsfh),
    'lambdar_log_mstar_error': dellogmintsfh,
    'z': z}
) 
#     'logmremnants' : logmremnants, 
#     'fluxscale' : fluxscale})
lambdar_masses.GAMA_ID = lambdar_masses.GAMA_ID.astype(int)
print(lambdar_masses.columns)

Index(['Unnamed: 0', 'Unnamed: 0.1', 'SDSS_ID', 'z lens', 'z src', 'GAMA_ID',
       'RA', 'DEC', 'STELLAR_MASS', 'Z'],
      dtype='object')
Index(['GAMA_ID', 'lambdar_log_mstar', 'lambdar_mstar',
       'lambdar_log_mstar_error', 'z'],
      dtype='object')


In [4]:
# Take the 
slacs_lambdar = pd.merge(slacs, lambdar_masses, how='left', on = 'GAMA_ID')
slacs_lambdar

,Unnamed: 0,Unnamed: 0.1,SDSS_ID,z lens,z src,GAMA_ID,RA,DEC,STELLAR_MASS,Z,lambdar_log_mstar,lambdar_mstar,lambdar_log_mstar_error,z
0,0,0,SDSSJ0912+0029,0.164,0.324,216398.0,138.02213,0.48366,4.438000e+11,0.1642,11.866503,7.353646e+11,0.105894,0.16419
1,1,1,SDSSJ1143−0144,0.106,0.402,136604.0,175.87349,-1.74167,2.866000e+11,0.1060,11.663817,4.611236e+11,0.113537,0.10605
2,2,2,SDSSJ1436−0000,0.285,0.805,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,3,SDSSJ1451−0239,0.125,0.520,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
# estimate Einstein radius for "thin lens" (point mass, Auger+) 
# and the two SIS equations (Zahid+ and Auger+)
slacs_lambdar_G216398 = slacs_lambdar[slacs_lambdar.GAMA_ID == 216398]
slacs_lambdar_G216398
# calculate mass at half effective radius (Auger+)
slacs_lambdar_G216398['m_half_re'] = 0.0011 * (slacs_lambdar_G216398.lambdar_mstar) ** (1.25)
# calculate theta_e_pm
M = slacs_lambdar_G216398.m_half_re
slacs_lambdar_G216398['D_lens'] = cosmo.angular_diameter_distance(slacs_lambdar_G216398.z).value
DL = slacs_lambdar_G216398.D_lens
DS = 2*slacs_lambdar_G216398.D_lens
slacs_lambdar_G216398['theta_e_pm'] = (M/(10**(8.09)))**(1/2) * ((DS - DL)/(DL*DS))**(1/2)
# create velocity disperions estimator (Zahid+)
def calculate_sigma (z, mass):
    mb = 10**(10.26)
    if z < 0.2: # SDSS fit
        sigma_b = 10**(2.073)
        if mass < mb:
            alpha = 0.403
        else:
            alpha = 0.293
        sigma = sigma_b * ( mass / mb ) ** (alpha)
    elif (z >= 0.2) & (z < 0.7): # SHELS fit
        sigma_b = 10**(2.071)
        alpha = 0.281
        sigma = sigma_b * ( mass / mb ) ** (alpha)
    else:
        print(f'Redshift out of range. {z}')
        sigma_b = 10**(2.071)
        alpha = 0.281
        sigma = sigma_b * ( mass / mb ) ** (alpha)
    return sigma
# calcualte velocity dispersion for slacs lens
# zahid+
z = slacs_lambdar_G216398.z.loc[0]
mass = slacs_lambdar_G216398.lambdar_mstar.loc[0]
slacs_lambdar_G216398['sigma_star_zahid'] = calculate_sigma(z = z, 
                                                      mass = mass)
# auger+
slacs_lambdar_G216398['sigma_star_auger'] = 10**2.34 * (mass/(10**11))**0.18
# calculate SIS Einstein radius estimates for each
# zahid+
slacs_lambdar_G216398['theta_e_veldisp_zahid'] = 206265 * 2 * np.pi * (slacs_lambdar_G216398.sigma_star_zahid)**2 / c**2
# auger+
slacs_lambdar_G216398['theta_e_veldisp_auger'] = 206265 * 2 * np.pi * (slacs_lambdar_G216398.sigma_star_auger)**2 / c**2
print(slacs_lambdar_G216398.theta_e_pm,
     slacs_lambdar_G216398.theta_e_veldisp_zahid,
     slacs_lambdar_G216398.theta_e_veldisp_auger)

0    2.344787
Name: theta_e_pm, dtype: float32 0    1.763497
Name: theta_e_veldisp_zahid, dtype: float64 0    1.415484
Name: theta_e_veldisp_auger, dtype: float64


/home/shawn/miniconda3/envs/astroconda/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/shawn/miniconda3/envs/astroconda/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/home/shawn/miniconda3/envs/astroconda/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_inde

In [30]:
slacs_lambdar_G216398[ ['SDSS_ID','z','lambdar_log_mstar','theta_e_pm', 'theta_e_veldisp_zahid', 'theta_e_veldisp_auger']]

,SDSS_ID,z,lambdar_log_mstar,theta_e_pm,theta_e_veldisp_zahid,theta_e_veldisp_auger
0,SDSSJ0912+0029,0.16419,11.866503,2.344787,1.763497,1.415484


In [27]:
# the slacs einstein radius is 1.63"

In [29]:
slacs_lambdar_G216398.lambdar_log_mstar

0    11.866503
Name: lambdar_log_mstar, dtype: float32

# 08/03/20 - Follow-up because G3882191 is also a SLACS/LinKS lens... Apparently it's not in this SLACS sample or there was a mistake somewhere. We determined that G3882191 was a SLACS lens because it is referenced as such in the HLA.
# Edit 08/13/20 - G3882191 was not a SLACS lens. It is SL2S and is a group.

In [5]:
lambdar_masses[lambdar_masses.GAMA_ID == 3882191]

,GAMA_ID,lambdar_log_mstar,lambdar_mstar,lambdar_log_mstar_error,z
118883,3882191,12.000241,1.000556e+12,0.110945,0.35562


# Edit 08/10/20

In [11]:
# estimate Einstein radius for "thin lens" (point mass, Auger+) 
# and the two SIS equations (Zahid+ and Auger+)
#slacs_lambdar_G216398 = slacs_lambdar[slacs_lambdar.GAMA_ID == 216398]
#slacs_lambdar_G216398
# calculate mass at half effective radius (Auger+)
slacs_lambdar['m_half_re'] = 0.0011 * (slacs_lambdar.lambdar_mstar) ** (1.25)
# calculate theta_e_pm
M = slacs_lambdar.m_half_re
slacs_lambdar['D_lens'] = cosmo.angular_diameter_distance(slacs_lambdar['z lens']).value
slacs_lambdar['D_source'] = cosmo.angular_diameter_distance(slacs_lambdar['z src']).value
DL = slacs_lambdar.D_lens
DS = slacs_lambdar.D_source
slacs_lambdar['theta_e_pm'] = (M/(10**(8.09)))**(1/2) * ((DS - DL)/(DL*DS))**(1/2)
# create velocity disperions estimator (Zahid+)
def calculate_sigma (z, mass):
    mb = 10**(10.26)
    if z < 0.2: # SDSS fit
        sigma_b = 10**(2.073)
        if mass < mb:
            alpha = 0.403
        else:
            alpha = 0.293
        sigma = sigma_b * ( mass / mb ) ** (alpha)
    elif (z >= 0.2) & (z < 0.7): # SHELS fit
        sigma_b = 10**(2.071)
        alpha = 0.281
        sigma = sigma_b * ( mass / mb ) ** (alpha)
    else:
        print(f'Redshift out of range. {z}')
        sigma_b = 10**(2.071)
        alpha = 0.281
        sigma = sigma_b * ( mass / mb ) ** (alpha)
    return sigma

def calculate_sigmas (z, mass):
    sigmas = []
    for i in range(len(z)):
        sigma = calculate_sigma(z[i], mass[i])
        sigmas.append(sigma)
    return sigmas
# calcualte velocity dispersion for slacs lens
# zahid+
z = slacs_lambdar.z#.loc[0]
mass = slacs_lambdar.lambdar_mstar#.loc[0]
slacs_lambdar['sigma_star_zahid'] = calculate_sigmas(z = z, 
                                                      mass = mass)
# auger+
slacs_lambdar['sigma_star_auger'] = 10**2.34 * (mass/(10**11))**0.18
# calculate SIS Einstein radius estimates for each
# zahid+
slacs_lambdar['theta_e_veldisp_zahid'] = 206265 * (DS-DL) / DS *  4 * np.pi * (slacs_lambdar.sigma_star_zahid)**2 / c**2
# auger+
slacs_lambdar['theta_e_veldisp_auger'] = 206265 * (DS-DL) / DS * 4 * np.pi * (slacs_lambdar.sigma_star_auger)**2 / c**2
print(slacs_lambdar.theta_e_pm,
     slacs_lambdar.theta_e_veldisp_zahid,
     slacs_lambdar.theta_e_veldisp_auger)

Redshift out of range. nan
Redshift out of range. nan
0    2.106289
1    2.393820
2         NaN
3         NaN
Name: theta_e_pm, dtype: float64 0    1.421641
1    1.724167
2         NaN
3         NaN
Name: theta_e_veldisp_zahid, dtype: float64 0    1.141091
1    1.537859
2         NaN
3         NaN
Name: theta_e_veldisp_auger, dtype: float64


In [14]:
slacs_lambdar[ ['GAMA_ID', 'z lens', 'z src', 'theta_e_pm', 'theta_e_veldisp_zahid', 'theta_e_veldisp_auger' ] ]

,GAMA_ID,z lens,z src,theta_e_pm,theta_e_veldisp_zahid,theta_e_veldisp_auger
0,216398.0,0.164,0.324,2.106289,1.421641,1.141091
1,136604.0,0.106,0.402,2.393820,1.724167,1.537859
2,NaN,0.285,0.805,NaN,NaN,NaN
3,NaN,0.125,0.520,NaN,NaN,NaN


In [15]:
DS/DL

0    1.675251
1    2.798073
2    1.771889
3    2.810027
dtype: float64

# Edit 08/13/20 - G3882191

In [7]:
mac = pd.read_csv(f'{csv_datapath}mac_latest.csv')
mac[mac.GAMA_ID == 3882191]

,Unnamed: 0,Unnamed: 0.1,GAMA_ID,RA,DEC,score,z,magphys_mstar,lambdar_log_mstar,lambdar_mstar,lambdar_log_mstar_error,m_half_re,sigma_star,D_lens,theta_e_pm,theta_e_sis
14,14,14,3882191.0,133.69397,-1.36032,54.0,0.3556,3.933000e+11,12.000241,1.000556e+12,0.110945,1.100764e+12,363.084577,986.079072,2.129984,1.905284


In [13]:
G3882191 = mac[mac.GAMA_ID == 3882191]
G3882191['theta_e_measured'] = 5.48
G3882191['z_source'] = 1.268
G3882191['D_source'] = cosmo.angular_diameter_distance(G3882191['z_source']).value

DL = G3882191.D_lens
DS_measured = G3882191.D_source
DS_assumed = 2*DL
mass = G3882191.lambdar_mstar
M = G3882191.m_half_re

G3882191['sigma_star_auger'] = 10**2.34 * (mass/(10**11))**0.18
sigma_auger = G3882191.sigma_star_auger
sigma_zahid = G3882191.sigma_star

# assuming D
G3882191['theta_e_pm_measured'] = (M/(10**(8.09)))**(1/2) * ((DS_measured - DL)/(DL*DS_measured))**(1/2)
G3882191['thete_e_veldisp_zahid_measured'] = 206265 * (DS_measured-DL) / DS_measured * 4 * np.pi * (sigma_zahid)**2 / c**2
G3882191['theta_e_veldisp_auger_assumed'] = 206265 * (DS_assumed-DL) / DS_assumed * 4 * np.pi * (sigma_auger)**2 / c**2
G3882191['theta_e_veldisp_auger_measured'] = 206265 * (DS_measured-DL) / DS_measured * 4 * np.pi * (sigma_auger)**2 / c**2

G3882191

/home/shawn/miniconda3/envs/astroconda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/shawn/miniconda3/envs/astroconda/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/shawn/miniconda3/envs/astroconda/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy 

,Unnamed: 0,Unnamed: 0.1,GAMA_ID,RA,DEC,score,z,magphys_mstar,lambdar_log_mstar,lambdar_mstar,...,theta_e_pm,theta_e_sis,theta_e_measured,z_source,D_source,sigma_star_auger,theta_e_pm_measured,thete_e_veldisp_zahid_measured,theta_e_veldisp_auger_assumed,theta_e_veldisp_auger_measured
14,14,14,3882191.0,133.69397,-1.36032,54.0,0.3556,3.933000e+11,12.000241,1.000556e+12,...,2.129984,1.905284,5.48,1.268,1679.507172,331.164238,1.935535,1.569745,1.581433,1.305871
